# HuggingFace Model → RunPod Network Volume

Colab で HuggingFace からモデルをダウンロードし、RunPod の Network Volume に S3 互換 API でアップロードする。

## Colab Secrets に以下を設定

1. 左サイドバーの 🔑 アイコンをクリック
2. 以下の Secrets を追加:
   - `HF_TOKEN`: HuggingFace アクセストークン
   - `RUNPOD_STORAGE_ACCESS_KEY_ID`: RunPod Storage の Access Key ID
   - `RUNPOD_STORAGE_SECRET_ACCESS_KEY`: RunPod Storage の Secret Access Key
3. 各 Secret の「Notebook access」をオンにする

## RunPod Storage 設定の取得

1. https://console.runpod.io/user/storage を開く
2. 対象 Volume の「S3 Compatible API Commands」の Example をコピー
   ```
   aws s3 ls --region xxx --endpoint-url https://s3api-xxx.runpod.io s3://your-volume-id/
   ```
3. 下のセルに REGION, ENDPOINT_URL, BUCKET を設定

In [ ]:
# ==================================================
# 設定
# ==================================================
from google.colab import userdata

# 認証情報
%env HF_TOKEN={userdata.get('HF_TOKEN')}
%env AWS_ACCESS_KEY_ID={userdata.get('RUNPOD_STORAGE_ACCESS_KEY_ID')}
%env AWS_SECRET_ACCESS_KEY={userdata.get('RUNPOD_STORAGE_SECRET_ACCESS_KEY')}

# HuggingFace モデル (複数指定可)
# https://huggingface.co/USER/REPOSITORY の USER/REPOSITORY を指定
HF_MODELS = [
    "Qwen/Qwen3-8B",
]

# RunPod Storage 設定
# https://console.runpod.io/user/storage の Example からコピー
# 例: aws s3 ls --region xxx --endpoint-url https://s3api-xxx.runpod.io s3://your-volume-id/
REGION = ""  # @param {type:"string"}
ENDPOINT_URL = ""  # @param {type:"string"}
BUCKET = ""  # @param {type:"string"}

In [ ]:
# 依存パッケージのインストール
!pip install -q huggingface_hub[hf_transfer]
!pip install -q awscli

In [ ]:
# モデルごとにダウンロード & アップロード
for model in HF_MODELS:
    local_dir = f"/content/models/{model}"
    dest_path = f"models/{model.split('/')[-1]}"

    print(f"\n{'='*60}")
    print(f"=== {model} ===")
    print(f"{'='*60}")

    # ダウンロード
    print(f"\n[1/3] Downloading {model}...")
    !HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download {model} --local-dir={local_dir}

    # 確認
    print(f"\n[2/3] Downloaded files:")
    !du -sh {local_dir}

    # アップロード
    print(f"\n[3/3] Uploading to s3://{BUCKET}/{dest_path}...")
    !aws s3 sync {local_dir} s3://{BUCKET}/{dest_path} \
        --region={REGION} \
        --endpoint-url={ENDPOINT_URL}

## トラブルシューティング

### 一部ファイルのアップロードに失敗した場合

`upload failed` が出た場合は checksum オプション付きで再実行:

In [ ]:
# チェックサム付きで再アップロード (失敗時用)
# for model in HF_MODELS:
#     local_dir = f"/content/models/{model}"
#     dest_path = f"models/{model.split('/')[-1]}"
#     !aws s3 sync {local_dir} s3://{BUCKET}/{dest_path} \
#         --region={REGION} \
#         --endpoint-url={ENDPOINT_URL} \
#         --checksum-algorithm=CRC32C

### Volume が大きく sync が失敗する場合

> fatal error: Error during pagination: The same next token was received twice: ...

`cp --recursive` を使用 (差分転送は不可):

In [ ]:
# sync が失敗する場合は cp --recursive を使用
# for model in HF_MODELS:
#     local_dir = f"/content/models/{model}"
#     dest_path = f"models/{model.split('/')[-1]}"
#     !aws s3 cp --recursive {local_dir} s3://{BUCKET}/{dest_path} \
#         --region={REGION} \
#         --endpoint-url={ENDPOINT_URL}

In [ ]:
# アップロード結果の確認
for model in HF_MODELS:
    dest_path = f"models/{model.split('/')[-1]}"
    print(f"=== {dest_path} ===")
    !aws s3 ls s3://{BUCKET}/{dest_path}/ \
        --region={REGION} \
        --endpoint-url={ENDPOINT_URL}